# 라이브러리 로드

In [1]:
import os


import tensorflow as tf
import tensorflow_datasets as tfds
import cv2
import imageio
from PIL import Image

from skimage import data
from skimage.data import coffee, rocket, astronaut, chelsea
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import numpy as np

from tensorflow.keras import Input, Model, layers
from tensorflow.python.keras import applications
from tensorflow.keras import losses, metrics, optimizers

import matplotlib.pyplot as plt

In [2]:
tfds.load("div2k/bicubic_x4")

{'train': <PrefetchDataset shapes: {hr: (None, None, 3), lr: (None, None, 3)}, types: {hr: tf.uint8, lr: tf.uint8}>,
 'validation': <PrefetchDataset shapes: {hr: (None, None, 3), lr: (None, None, 3)}, types: {hr: tf.uint8, lr: tf.uint8}>}

# Bicubic interpolation, SRGAN 비교

In [3]:
model_path = os.getenv('HOME')+'/aiffel/super_resolution/srgan_G.h5'
model = tf.keras.models.load_model(model_path, compile=False)

In [4]:
class SuperResolution:
    def __init__(self, image, model, show=True):
        self.show = show
        self.srgan = model
        self.image = self.resize_image(image)
        self.interpolation_image = self.make_interpolation(self.image)
        self.srgan_image = self.make_srgan(self.interpolation_image)
        self.show_srgan()
    
    
    def resize_image(self, image):
        w = (image.shape[0] // 4) * 4
        h = (image.shape[1] // 4) * 4
        return cv2.resize(image, dsize=(h, w))
    
    
    def interpolation_xn(self, image, n):
        img_shape = image.shape
        downsample = cv2.resize(
            image,
            dsize=(img_shape[1]//n, img_shape[0]//n)
        )
        upsample = cv2.resize(
            downsample,
            dsize=(img_shape[1], img_shape[0]),
            interpolation=cv2.INTER_CUBIC
        )
        return upsample

    
    def make_interpolation(self, image):
        image4 = self.interpolation_xn(image, 4)

        images = [image, image4]
        titles = ["HR", "x4"]
        
        psnr = [round(peak_signal_noise_ratio(image, i), 3) for i in images]
        ssim = [round(structural_similarity(image, i, multichannel=True), 3) for i in images]
        
        if self.show:
            plt.figure(figsize=(16,10))
            for i in range(2):
                plt.subplot(2,1,i+1)
                plt.imshow(images[i])
                plt.title(titles[i] + f" [{psnr[i]}/{ssim[i]}]", fontsize=20)
        
        return images[-1]
    
    
    def make_srgan(self, image):
        downsample = cv2.resize(image, dsize=(image.shape[1]//4, image.shape[0]//4))
        image = tf.cast(downsample[np.newaxis, ...], tf.float32)
        sr = self.srgan.predict(image)
        sr = tf.clip_by_value(sr, 0, 255)
        sr = tf.round(sr)
        sr = tf.cast(sr, tf.uint8)
        return np.array(sr)[0]
    
    
    def show_srgan(self):
        plt.figure(figsize=(16,10))
        origin = self.image
        img = self.srgan_image
        psnr = round(peak_signal_noise_ratio(origin, img), 3)
        ssim = round(structural_similarity(origin, img, multichannel=True), 3)
        if self.show:
            plt.subplot(2,1,1)
            plt.imshow(img)
            plt.title("SRGAN x4"  + f" [{psnr}/{ssim}]", fontsize=20)
            

## 저해상도 이미지 복원

In [6]:
cat = chelsea()
sr = SuperResolution(cat, model)

<img src="https://i.ibb.co/5cPwZVK/2021-11-25-17-00-15.png" width=400></img>

In [7]:
coffee_img = coffee()
sr = SuperResolution(coffee_img, model)

<img src="https://i.ibb.co/3mC4SZn/2021-11-25-17-00-11.png" width=400></img>

In [10]:
rocket_img = rocket()
sr = SuperResolution(rocket_img, model)

<img src="https://i.ibb.co/D420qXG/2021-11-25-17-00-07.png" width=400></img>

In [11]:
astronaut_img = astronaut()
sr = SuperResolution(astronaut_img, model)

<img src="https://i.ibb.co/zhpn5py/2021-11-25-17-00-03.png" width=300></img>

## 고해상도 이미지 복원

In [12]:
newyork = np.asarray(Image.open('newyork.jpg'))
sr = SuperResolution(newyork, model)

<img src="https://i.ibb.co/7zyMNxN/2021-11-25-18-13-34.png" width=300></img>

In [13]:
waterfall = np.asarray(Image.open('duomo.jpg'))
sr = SuperResolution(waterfall, model)

<img src="https://i.ibb.co/QcbVbp4/2021-11-25-18-13-19.png" width=300></img>

# 저해상도 gif 동영상, 고해상도 변환

In [19]:
class GIFConvert:
    def __init__(self, gif_path, model):
        self.gif_path = gif_path
        self.model = model
        self.gif = cv2.VideoCapture(gif_path)
        self.frames = self.make_frames()
        self.bicubics, self.srgan = self.make_new_gifs()
        self.save_gifs()
    
    def make_frames(self):
        frames = []
        while True:
            check, frame = self.gif.read()
            if check:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frames.append(frame)
            else:
                break
        return frames
    
    def make_new_gifs(self):
        bicubics_images = []
        srgan_images = []

        for frame in self.frames:
            sr = SuperResolution(frame, self.model, show=False)
            bicubics_images.append(sr.interpolation_image)
            srgan_images.append(sr.srgan_image)
        return bicubics_images, srgan_images
    
    def save_gifs(self):
        imageio.mimsave("원본.gif", self.frames)        
        imageio.mimsave("저해상도.gif", self.bicubics)
        imageio.mimsave("SRGAN 복원.gif", self.srgan)

In [20]:
converter = GIFConvert("book_lr.gif", model)

<h4>원본</h4>
<img src="https://i.ibb.co/Mf2wxtJ/image.gif" width="380">
<h4>Interpolation 복원</h4>
<img src="https://i.ibb.co/g4wMBCQ/image.gif" width="380">
<h4>SRGAN 복원</h4>
<img src="https://i.ibb.co/55C4WC5/SRGAN.gif" width="380">


- 고해상도 이미지로 훈련된 모델이라서 그런지 동일한 사진은 아니지만 해상도가 높은 사진에서 더 좋은 성능을 보임을 확인할 수 있었습니다.